# Scan Repositories

This notebook will scan repositories in order to collect details about `wontfix` label usage

In [2]:
import csv
from typing import Tuple, List

import requests, os
import pandas as pd
from ratelimit import limits, sleep_and_retry

# Constants
GITHUB_TOKEN = os.getenv('GITHUB_TOKEN')

In [3]:
df = pd.read_csv('../data/repos.csv', index_col='id')

# Convert these strings of semicolon-seperated values into a proper array
df['labels'] = df['labels'].str.split(';')
df['topics'] = df['topics'].apply(lambda x: [] if pd.isna(x) else x.split(';'))

df.head()

,name,isFork,commits,branches,releases,forks,mainLanguage,defaultBranch,license,homepage,...,isDisabled,isLocked,languages,labels,topics,wontfixType,updatedIssuesTotal,updatedPullsTotal,wontfixIssues,wontfixPulls
id,,,,,,,,,,,,,,,,,,,,,


## Fetch from GitHub

Now we can go through each repo and fetch `wontfix` issues from GitHub

### Steps

1. Loop through each repo in master list (repos.csv)
2. Get issues for each repo 
    - Use: `fetch_issues_for_repo()`
3. Separate issues from PRs
    - Use: `separate_issues_from_prs()`
4. Record the total issues and wontfix issues on repos.csv
    - Use: `len(issues)` to get wontfix issues
    - Use: `fetch_total_issues_and_prs_for_repo()`
5. Record the total PRs and wontfix PRs on the repos.csv
    - Use: `len(pulls)` to get wonfix pull request total
    - Use results from 4b above
6. Record each issue onto the issues.csv
7. Record each PR onto the pull-requests.csv
8. Check if loop should continue, repeating as necessary

In [4]:
@sleep_and_retry
@limits(calls=4950, period=3600) # 5000 calls per hour
def check_rate_limit():
    """
    Empty function to check the rate limit of the GitHub API
    """
    return

In [5]:
def fetch_issues_for_repo(repo_name: str, label: str = 'wontfix', per_page: int = 100, start_page: int = 1, issue_state: str = 'all') -> list:
    """
    Fetches issues from GitHub API for a specific repo with a given label. Paginates through requests until all issues have been collected. Returns a list of
    all matched issues. 
    :param repo_name: 
    :param label: 
    :param per_page: 
    :param start_page: 
    :param issue_state: 
    :return: list of all_issues
    """
    headers = {
        'Authorization': f'Bearer {GITHUB_TOKEN}',
        'Accept': 'application/vnd.github+json'
    }

    current_page = start_page
    repo_name = repo_name
    label = label
    all_issues = []

    # GitHub API URL for fetching issues from the repository
    issues_url = f'https://api.github.com/repos/{repo_name}/issues'

    while True:
        params = {
            'labels': label,
            'state': issue_state,  # Change to 'open' if you only want open issues
            'per_page': per_page,  # Adjust per_page to your needs, maximum is 100
            'page': current_page
        }
        
        # Send API Request
        check_rate_limit()
        response = requests.get(issues_url, headers=headers, params=params)

        # Check for successful response
        if not response.ok:
            print('Failed to fetch issues:', response.content)
            break
        
        issues = response.json()
        all_issues += issues
        
        link_header = response.headers['Link'] if 'Link' in response.headers else ""
        current_page += 1
        if 'rel="next"' not in link_header:
            break # End of pages
            
    return all_issues

In [6]:
def separate_issues_from_prs(items: list) -> Tuple[List[dict], List[dict]]:
    """
    Separate a returned list of issues into a list of Pull Requests (PRs) and "proper" Issues (which are not PRs)
    Technically all PRs are issues, so the GitHub API returns PRs as an issue, and that is what necessitates this step.
    :param items: 
    :return: (PRs, Issues)
    """
    prs = []
    issues = []
    for item in items:
        if 'pull_request' in item:
            prs.append(item)
        else:
            issues.append(item)
    return issues, prs


In [7]:
results = fetch_issues_for_repo('jacurtis/wontfix-research', per_page=100)
res_issues, res_prs = separate_issues_from_prs(results)
        
print(f"Issues: {len(res_issues)}")
print(f"PRs: {len(res_prs)}")

Issues: 0
PRs: 0


In [8]:
def _extract_total_count(link_header: str) -> int:
    """
    Extracts the total number of pages from a link header on a GitHub API response.
    If you only use per_page=1, then this can easily map to the total number of issues/prs with a single request
    :param link_header: 
    :return total_count: 
    """
    if link_header:
        parts = link_header.split(',')
        for part in parts:
            if 'rel="last"' in part:
                return int(part.split("&page=")[1].split(">")[0])
    return 0

def fetch_total_issues_and_prs_for_repo(repo_name: str) -> (int, int):
    """
    Fetches the total number of issues and pull requests for a given repository
    :param repo_name: 
    :return total_issues, total_pulls:
    """
    issues_url = f'https://api.github.com/repos/{repo_name}/issues'
    pulls_url = f'https://api.github.com/repos/{repo_name}/pulls'
    
    headers = {
        'Authorization': f'Bearer {GITHUB_TOKEN}',
        'Accept': 'application/vnd.github+json'
    }
    
    params = {
        'state': 'all',
        'per_page': 1
    }
    
    check_rate_limit()
    response_pulls = requests.get(pulls_url, headers=headers, params=params)
    check_rate_limit()
    response_issues = requests.get(issues_url, headers=headers, params=params)
    
    header_pulls_link = response_pulls.headers.get('Link', None)
    header_issues_link = response_issues.headers.get('Link', None)
    
    total_pulls = _extract_total_count(header_pulls_link)
    total_issues = _extract_total_count(header_issues_link) - total_pulls
    
    return total_issues, total_pulls

In [9]:
# fetch_total_issues_and_prs_for_repo('rzwitserloot/lombok')

In [10]:
def save_issues_to_csv(issues: List[dict], filename: str, repo_name: str, repo_id: int) -> None:
    """
    Save a list of issues to a CSV file
    :param issues: 
    :param filename: 
    :return: 
    """
    with open(filename, 'a') as file:
        writer = csv.writer(file)
        for issue in issues:
            # pprint.pprint(issue)
            issue_type = 'Pull' if 'pulls' in filename else 'Issue'
            print(f"{issue_type}: {repo_name} #{issue['number']} - {issue['title']}")
            row = [
                issue['id'], 
                issue['title'], 
                issue['number'], 
                repo_id, 
                repo_name, 
                [label['name'] for label in issue['labels']], 
                issue['state'], 
                issue['comments'], 
                issue['created_at'], 
                issue['updated_at'], 
                issue['closed_at'], 
                issue['body'] if 'body' in issue else None, 
                issue['user']['login'] if 'user' in issue else None, 
                issue['user']['id'] if 'user' in issue else None, 
            ]
            writer.writerow(row)


In [11]:
def update_repos(start_line: int, end_line: int, source_filename: str = '../data/raw/wontfix.csv', dest_filename: str = '../data/repos.csv', wontfix_type: str = 'wontfix') -> None:
    """
    Given a start line and an end line, go down the repos.csv file taking the repo from each line and
    updating the pull/issue totals and extracting all the issues for that repository.
    Note: Line numbers are zero-indexed, so line 1 is the 2nd line in the file. This works because the first line is the header.
    :param start_line: 
    :param end_line: 
    :param source_filename: 
    :param dest_filename: 
    :param wontfix_type: 
    :return None: 
    """
    with open(source_filename, 'r') as source_file:
        source_reader = csv.reader(source_file)
        with open(dest_filename, 'a') as dest_file:
            dest_writer = csv.writer(dest_file)
            for i, row in enumerate(source_reader):
                if start_line <= i <= end_line:
                    repo_id = row[0] # Repo ID is in the first column
                    repo_name = row[1] # Repo name is in the second column
                    row.append(wontfix_type) # static value to indicate the text used for the wontfix label in this repo
                    
                    # Get the total issues and PRs for the repo
                    total_issues, total_pulls = fetch_total_issues_and_prs_for_repo(repo_name)
                    row.append(total_issues)
                    row.append(total_pulls)
                    
                    # Get the issues for the repo
                    results = fetch_issues_for_repo(repo_name, label=wontfix_type, per_page=100)
                    res_issues, res_prs = separate_issues_from_prs(results)
                    row.append(len(res_issues))
                    row.append(len(res_prs))
                    
                    # Write the new row to the file
                    dest_writer.writerow(row)

                    # Write the issues and PRs to separate files
                    # TODO: Write the issues to a file
                    save_issues_to_csv(res_issues, '../data/issues.csv', repo_name, repo_id)
                    # TODO: Write the PRs to a file
                    save_issues_to_csv(res_prs, '../data/pulls.csv', repo_name, repo_id)
                

In [33]:
update_repos(1700, 1779)

Issue: ggerganov/llama.cpp #1846 - Applying lora with CUDA crashes with failed assertion
Issue: ggerganov/llama.cpp #741 - Add OpenCL Support
Issue: ggerganov/llama.cpp #650 - How do i download the models? 
Issue: ggerganov/llama.cpp #554 - Windows defender finds a virus in current master branch
Issue: ggerganov/llama.cpp #209 - Command line script usage
Issue: ggerganov/llama.cpp #200 - Running " python3 convert-pth-to-ggml.py models/7B/ 1 " and running out of RAM
Issue: ggerganov/llama.cpp #144 - Interactive mode does not work
Issue: ggerganov/llama.cpp #140 - Only show prompt and response 
Issue: ggerganov/llama.cpp #121 - llama_model_load: llama_model_load: unknown tensor '' in model file
Issue: ggerganov/llama.cpp #111 - Make a tag/release
Issue: ggerganov/llama.cpp #104 - Anyplan to make CodeGenCPP?
Issue: ggerganov/llama.cpp #94 - Segfault using the chat like interface on the 65B parameterized model
Issue: ggerganov/llama.cpp #76 - .pth to .ggml Out of Memory
Issue: ggerganov/ll